# Data Extraction  

The actual scorecard for the match below can be viewed at [Cricinfo webpage.](https://www.espncricinfo.com/series/8048/scorecard/335982/royal-challengers-bangalore-vs-kolkata-knight-riders-1st-match-indian-premier-league-2007-08)


In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os, glob, yaml

data_dir='../datasets/ipl/ipl/'

In [2]:
f = '335982.yaml'
fil=os.path.join(data_dir,  f )
data=yaml.load(open(fil))

In [3]:
print ( data.keys() )
print ()
print ( data['meta'] )
print ()
data['info']

dict_keys(['meta', 'info', 'innings'])

{'data_version': 0.9, 'created': datetime.date(2011, 5, 6), 'revision': 2}



{'city': 'Bangalore',
 'competition': 'IPL',
 'dates': [datetime.date(2008, 4, 18)],
 'gender': 'male',
 'match_type': 'T20',
 'outcome': {'by': {'runs': 140}, 'winner': 'Kolkata Knight Riders'},
 'overs': 20,
 'player_of_match': ['BB McCullum'],
 'teams': ['Royal Challengers Bangalore', 'Kolkata Knight Riders'],
 'toss': {'decision': 'field', 'winner': 'Royal Challengers Bangalore'},
 'umpires': ['Asad Rauf', 'RE Koertzen'],
 'venue': 'M Chinnaswamy Stadium'}

In [38]:
list(data['info']['outcome']['by'].keys())[0]
list(data['info']['outcome']['by'].values())[0]

140

In [52]:
def print_scorecard(f, data_dir=data_dir):
    
    fil=os.path.join(data_dir,  f )
    data=yaml.load(open(fil))

    batting_card = {}    
    for inn in range(2):
        inn_name = list(data['innings'][inn].keys())[0]

        batting_card_inn={}
        batting_card_inn[inn_name] = {'0':' ', '1':' ', '2':' ', '3':' ', '4':' ', '6':' ', 'R':' ', 'B':' '}
        
        runs_bowler={}
        runs_extra = 0
        runs_total = 0
        wkts       = 0

        for delivery in list(data['innings'][inn].values())[0]['deliveries']:
            deliv = list(delivery.items())[0]
            
            ball     = deliv[0]
            batsman  = deliv[1]['batsman'].strip()
            bowler   = deliv[1]['bowler'].strip()
            runs_bat = deliv[1]['runs'].get('batsman', 0)
            runs_ext = deliv[1]['runs'].get('extras',  0)
            runs_tot = deliv[1]['runs'].get('total',   0)
            
            if batsman not in batting_card_inn:
                batting_card_inn[batsman] = {'0':0, '1':0, '2':0, '3':0, '4':0, '6':0, 'R':0, 'B':0}

            batting_card_inn[batsman][str(runs_bat)] = batting_card_inn[batsman][str(runs_bat)]+1
            batting_card_inn[batsman]['R'] += runs_bat
            batting_card_inn[batsman]['B'] += 1 # remove the ball later if it's a wide
            
            if 'extras' in deliv[1]:
                if 'wides' in deliv[1]['extras']:
                    batting_card_inn[batsman]['B'] -= 1 # remove the ball from batsman's account

            runs_extra += runs_ext
            runs_total += runs_tot
            
            if deliv[1].get('wicket', None):
                wkts += 1

        batting_card_inn['Total_inn'+str(inn+1)] = {'0':' ', '1':' ', '2':' ', '3':' ', '4':' ', '6':' ',
                                                    'R':str(runs_total)+'-'+str(wkts), 'B':' '}
        batting_card_inn['  '] = {'0':' ', '1':' ', '2':' ', '3':' ', '4':' ', '6':' ', 'R':' ', 'B':' '}        
        batting_card.update(batting_card_inn)
    
    df = pd.DataFrame(batting_card).transpose()
    df.style.set_caption("Title")

    print (data['info']['teams'][0] ,' vs ', data['info']['teams'][1], ', ', data['info']['dates'][0].strftime('%Y-%m-%d'))
    print ( 'Toss \t\t\t', data['info']['toss']['winner'], 'Decided to', data['info']['toss']['decision'])
    print ('Result \t\t\t', data['info']['outcome']['winner'], 'won by ', 
           list(data['info']['outcome']['by'].values())[0], list(data['info']['outcome']['by'].keys())[0])
    print ('Player of the Match\t', data['info']['player_of_match'][0])

        
    display(df)
    
    return df
    
f = '335982.yaml'

df = print_scorecard(f)

Royal Challengers Bangalore  vs  Kolkata Knight Riders ,  2008-04-18
Toss 			 Royal Challengers Bangalore Decided to field
Result 			 Kolkata Knight Riders won by  140 runs
Player of the Match	 BB McCullum


,0,1,2,3,4,6,R,B
1st innings,,,,,,,,
SC Ganguly,8,2,0,0,2,0,10,12
BB McCullum,23,22,9,0,10,13,158,73
RT Ponting,9,8,1,0,1,1,20,20
DJ Hussey,4,6,1,0,1,0,12,12
Mohammad Hafeez,1,1,0,0,1,0,5,3
Total_inn1,,,,,,,222-3,
,,,,,,,,
2nd innings,,,,,,,,
R Dravid,1,2,0,0,0,0,2,3
